# Préparation du jeu de données PubChem

## Ouverture et analyse des données

### Ouverture du jeu 

In [1]:
import h5py

dataset_location = "../data/riken_v2.h5"
minimal_dataset_location = "../data/minimal_dataset.h5"
train_dataset_location = "../data/train_set.h5"
test_dataset_location = "../data/test_set.h5"

# On charge le jeu de données original (en lecture seule)
original_dataset = h5py.File(dataset_location, 'r')

/home/etudiant/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Analyse des données

#### Affichage des clés (colonnes)

In [2]:
import pandas as pd
from IPython.display import display
import numpy as np


# Affichage des clés du fichier
print("Keys : ")
for key in original_dataset.keys():
    print(key)

Keys : 
pubchem_id
riken_coords
amasses
anums


#### Définition d'une fonction pour afficher les données d'une molécule à partir de son indice

In [3]:
def affichage_donnees_molecule(dataset, index):
    
    print()
    print()
    print("index molécule "+str(index))
    
    print()
    print("id pubchem : "+str(dataset["pubchem_id"][index]))
    
    print()
    coords = dataset["riken_coords"][index].reshape(-1,3)    
    df = pd.DataFrame(columns = ['numéro atomique', 'masse atomique', 'x', 'y', 'z'],
                      data = np.array([dataset["anums"][index], 
                              dataset["amasses"][index], 
                              coords[:, 0],
                              coords[:, 1],
                              coords[:, 2]]).T)
    
    display(df)

affichage_donnees_molecule(original_dataset, 15000)





index molécule 15000

id pubchem : 60146483



,numéro atomique,masse atomique,x,y,z
0,6.0,12.010700,1.4203,-1.0174,-0.4915
1,6.0,12.010700,10.2933,0.2510,-0.6398
2,6.0,12.010700,2.3897,-0.1447,-0.2215
3,6.0,12.010700,3.2027,0.5911,-1.2537
4,6.0,12.010700,7.7292,0.4491,-0.7290
5,6.0,12.010700,9.0948,1.1248,-0.9869
6,6.0,12.010700,5.4259,1.0577,-1.7704
7,6.0,12.010700,7.3375,1.6467,-2.7848
8,7.0,14.006700,4.6246,0.5095,-0.9391
9,7.0,14.006700,5.2373,1.7674,-2.9609


#### Définition d'une fonction pour charger le jeu de données complet comme DataFrame Pandas

In [ ]:
def chargement_database_df(dataset):
    
    df = pd.DataFrame(columns = ["pubchem id", "numéros atomiques", "masses atomiques", "coordonnées"],
                      data = np.array([dataset["pubchem_id"], 
                              dataset["anums"],
                              dataset["amasses"],
                              dataset["riken_coords"]]).T)

    return df

database_df = chargement_database_df(original_dataset)

#### Affichage des premières valeurs du jeu de données complet

In [ ]:
database_df.head()

#### Description des données

In [ ]:
database_df.describe()

### Travail sur un jeu de données fictif de même structure mais de petite taille pour expérimentation

#### Génération du jeu de données fictives

In [5]:
import numpy as np
from random import randint


def generer_seq(taille, a, b):
    return np.random.randint(a, b, taille)
    
def generer_dataset_fictif(taille, nb_atomes_max_mol):
    dataset = {}
    
    pubchem_id = []
    anums = []
    amasses = []
    riken_coords = []

    for i in range(taille):
        pubchem_id.append(i)

        nb_atomes = randint(1, nb_atomes_max_mol)
        anums.append(generer_seq(nb_atomes, 1, 100))
        amasses.append(generer_seq(nb_atomes, 1, 300))
        riken_coords.append(generer_seq(nb_atomes*3, -100, 100))
    
    dataset["pubchem_id"] = pubchem_id
    dataset["anums"] = anums
    dataset["amasses"] = amasses
    dataset["riken_coords"] = riken_coords
        
    return dataset

dataset_fictif = generer_dataset_fictif(2 , 3)
        
    
affichage_donnees_molecule(dataset_fictif, 1)
    



index molécule 1

id pubchem : 1



,numéro atomique,masse atomique,x,y,z
0,99,283,10,-43,57
1,99,76,-21,28,79


### Travail sur un sous-ensemble du jeu de données pour expérimentation

#### Création d'un fichier h5 contenant les n premiers exemples du jeu original

In [7]:
import time

def creer_h5_ss_ensemble(original_dataset_filename, dataset_dest_filename, taille):
    
    start_time = time.time()

    # Ouverture du dataset d'entrée en lecture seule
    original_dataset =  h5py.File(original_dataset_filename, 'r')
    
    # Création du dataset minimal de sortie
    dataset_reduit_h5 = h5py.File(dataset_dest_filename, 'w')
    
    try:
        # Définition du type spécial
        varlen_floatarray = h5py.special_dtype(vlen=np.dtype("float32"))

        print("Copie des données...")
        
        # Copie des premières valeurs de la colonne pubchem_id
        dataset_reduit_h5.create_dataset("pubchem_id", shape=(taille,),
                                       dtype=np.int32, compression="gzip", 
                                       chunks=True, maxshape=(None,), 
                                       data=original_dataset["pubchem_id"][:taille])

        # Copie des premières valeurs des autres colonnes
        cols = ["anums", "amasses", "riken_coords"]  
        for col_name in cols:
            dataset_reduit_h5.create_dataset(col_name, shape=(taille,),
                                           dtype=varlen_floatarray, compression="gzip", 
                                           chunks=True, maxshape=(None,), 
                                           data=original_dataset[col_name][:taille])

        dataset_reduit_h5.flush()
        print("Jeu minimal enregistré avec succès")
        print("--- %s seconds ---" % (time.time() - start_time))
        
    finally:
        dataset_reduit_h5.close()


        

        
creer_h5_ss_ensemble(dataset_location, minimal_dataset_location, 2)




Copie des données...
Jeu minimal enregistré avec succès
--- 0.007838726043701172 seconds ---


#### Définition d'une fonction pour séparer les données en un jeu de test et un jeu d'entraînement

In [5]:
from sklearn.model_selection import train_test_split
import time

def split_jeux(original_dataset_filename, train_dataset_filename, test_dataset_filename, test_size=0.1):
    """ Fonction prenant un dataset h5 en entrée et le séparant en un jeu d'entraînement et un jeu de test,
    qu'elle enregistre"""
    

    # On charge le jeu de données original (en lecture seule)
    original_dataset = h5py.File(original_dataset_filename, 'r')  

    # On créé un tableau numpy contenant toutes les données
    print("Loading the data...")
    data = np.array([np.int32(original_dataset["pubchem_id"]), 
              np.asarray(original_dataset["anums"]),
              np.asarray(original_dataset["amasses"]),
              np.asarray(original_dataset["riken_coords"])]).T
    
    # On fait appel à scikit learn pour séparer les données de façon aléatoire
    print("Separation of the data...")
    train_set, test_set = train_test_split(data, test_size=test_size, random_state = 12)
    
    
    # On créé deux fichiers h5
    print("Creating h5 files...")
    train_set_h5 = h5py.File(train_dataset_filename, 'w')
    test_set_h5 = h5py.File(test_dataset_filename, 'w')

    try:
        # Définition du type pour les tableaux de floats
        varlen_floatarray = h5py.special_dtype(vlen=np.dtype("float32"))

        # Calcul de la taille des deux datasets de sortie
        train_size = len(train_set)
        test_size = len(test_set)
        
        col_names = ["pubchem_id", "anums", "amasses", "riken_coords"]
        datatypes = [np.int32, varlen_floatarray, varlen_floatarray, varlen_floatarray]

        train_datasets = []
        test_datasets = []
        
        # On créé les quatre datasets (un par colonne) par fichier de sortie
        print("Creating datasets...")
        for i in range(4):
            
            train_datasets.append(train_set_h5.create_dataset(col_names[i], shape=(train_size,),
                                       dtype=datatypes[i], compression="gzip", 
                                       chunks=True, maxshape=(None,)))

            test_datasets.append(test_set_h5.create_dataset(col_names[i], shape=(test_size,),
                                       dtype=datatypes[i], compression="gzip", 
                                       chunks=True, maxshape=(None,)))

            
            
        # On écrit le dataset d'entraînement en mémoire
        print("Writing train set in memory")
        for i in range(train_size):
            j=0
            for col_name in col_names:
                train_datasets[j][i] = train_set[i][j]
                j += 1
        
        # On écrit le dataset de test en mémoire
        print("Writing test set in memory")
        for i in range(test_size):
            j=0
            for col_name in col_names:
                test_datasets[j][i] = test_set[i][j]
                j += 1


        
        # On écrit les datasets sur le disque
        print("Writing train set to disk...")
        train_set_h5.flush()

        print("Writing test set to disk...")
        test_set_h5.flush()
        
        print("Succesful creation of a train set and a test set")

        
    finally:
        # Closing the files
        original_dataset.close()
        train_set_h5.close()
        test_set_h5.close()


In [6]:

split_jeux(minimal_dataset_location, train_dataset_location, test_dataset_location)


Loading the data...
Separation of the data...
Creating h5 files...
Creating datasets...
Writing train set in memory
Writing test set in memory
Writing train set to disk...
Writing test set to disk...
Succesful creation of a train set and a test set


In [30]:


# Fermeture du jeu de données original
original_dataset_file.close()

NameError: name 'original_dataset_file' is not defined